## TOC:
* [Files](#this-is-to-create-the-db)
* [Code](#code-to-create-database)
    * [Headlines Schema](#headlines-schema)
    * [SP500 Schema](#sp500-schema)
    * [Finbert](#finbert-related-stuff)
    * [Finbert 10ks](#10k-finbert)
    * [VIX](#vix)
    * [VIX Preds](#vix-preds)
    * [Finbert All Tags](#finbert-all-scores)

# *THIS IS TO CREATE THE DB*
You can download the already made DB file `financial_news.db` from the sharepoint 

[practicum folder](https://gtvault-my.sharepoint.com/:f:/g/personal/ltupac3_gatech_edu/Eg2gLDzQ8H1JoWUrUIq1G04BPkOXMyxmhgcoL84Q58-5dg?e=80dziH)

[db file](https://gtvault-my.sharepoint.com/:u:/g/personal/ltupac3_gatech_edu/Edi6YX6MKPxMud1e5maTIjsBo04ISTst1j7uoxeSVH2OBA?e=XQD3Ed)

In [1]:
import duckdb
import pandas as pd 
import os 
import xml.etree.ElementTree as ET

# Code to create database
Instructions:
1. For individual files (company_info_news.txt, volume_news.csv, etc.), copy the relative path to the respective variable below
2. For headline data, put the relative path to the folder housing the ticker folders.
    - Example: MultiCap_News/HEADLINES houses the individual ticker folders. 
    - The code will recursively pick up the files from there

# Headlines Schema

In [ ]:
con = duckdb.connect("financial_news.db")

company_txt_path = 'MultiCap_News\\company_info_news.txt'
# volume_news_path = 'MultiCap_News\\volume_news.csv'
volume_news_path = 'Headlines_Data\\volume.csv'
# pricing_news_path = 'MultiCap_News\\pricing_news.csv'
pricing_news_path = 'Headlines_Data\\pricing.csv'
multicap_headlines = 'MultiCap_News\\HEADLINES'
headline_august24_path = 'HEADLINES_August24'

In [ ]:
# create the DDLs and indexes
ddl_statements = [
    "CREATE SCHEMA IF NOT EXISTS Headlines;",
    """
    CREATE TABLE IF NOT EXISTS headlines.Articles (
        guid TEXT,
        ticker TEXT,
        description TEXT,
        article_link TEXT,
        article_pubDate TIMESTAMP,
        article_title TEXT,
        language TEXT,
        lastBuildDate TIMESTAMP,
        link TEXT,
        title TEXT,
        PRIMARY KEY (guid, ticker)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS headlines.Company_Info_News (
        ticker TEXT PRIMARY KEY,
        name TEXT,
        subindustry TEXT
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS headlines.Pricing_News (
        trading_day_date DATE,
        ticker TEXT,
        price FLOAT,
        PRIMARY KEY (trading_day_date, ticker)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS headlines.Volume_News (
        trading_day_date DATE,
        ticker TEXT,
        volume INT,
        PRIMARY KEY (trading_day_date, ticker)
    );
    """,
    ############ Gold Layer ############
    """
    CREATE TABLE IF NOT EXISTS headlines.Market_Data_Daily_Processing (
    trading_day_date DATE,
    ticker TEXT,
    price FLOAT,
    volume INT,
    PRIMARY KEY (trading_day_date, ticker)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS headlines.Market_Data_Headlines (
    trading_day_date DATE,
    ticker TEXT,
    price FLOAT,
    volume INT,
    headline_count INT,
    PRIMARY KEY (trading_day_date, ticker)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS headlines.Trading_Calendar (
    trading_date DATE PRIMARY KEY
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS headlines.Articles_Trading_Day (
    guid TEXT,
    ticker TEXT,
    mapped_trading_date DATE,
    description TEXT,
    article_link TEXT,
    article_pubDate TIMESTAMP,
    article_title TEXT,
    language TEXT,
    lastBuildDate TIMESTAMP,
    link TEXT,
    title TEXT,
    PRIMARY KEY (guid, ticker)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS headlines.Market_Article_Summary (
    trading_date DATE PRIMARY KEY,
    article_count INT
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS headlines.Daily_Price_Movement (
    trading_date DATE,
    ticker TEXT,
    close_price FLOAT,
    next_trading_day DATE,
    close_price_next FLOAT,
    price_change FLOAT,
    price_change_percentage FLOAT,
    PRIMARY KEY (trading_date, ticker),
    FOREIGN KEY (trading_date) REFERENCES headlines.trading_calendar(trading_date),
    FOREIGN KEY (next_trading_day) REFERENCES headlines.trading_calendar(trading_date)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS headlines.Weekly_Price_Movement (
    trading_week_start DATE,
    ticker TEXT,
    close_price_start FLOAT,
    trading_week_end DATE,
    close_price_end FLOAT,
    price_change FLOAT,
    price_change_percentage FLOAT,
    PRIMARY KEY (trading_week_start, ticker),
    FOREIGN KEY (trading_week_start) REFERENCES headlines.trading_calendar(trading_date),
    FOREIGN KEY (trading_week_end) REFERENCES headlines.trading_calendar(trading_date)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS headlines.extreme_price_movements (
    trading_date DATE,
    ticker TEXT,
    close_price FLOAT,
    price_change FLOAT,
    price_change_percentage FLOAT,
    movement_type TEXT,  -- Drop|Surge
    PRIMARY KEY (trading_date, ticker)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS headlines.articles_extreme_drops (
    trading_date DATE,
    ticker TEXT,
    guid TEXT,
    mapped_trading_date DATE, 
    title_sentiment_score FLOAT,
    title_sentiment_label TEXT,
    description_sentiment_score FLOAT,
    description_sentiment_label TEXT,
    PRIMARY KEY (trading_date, ticker, guid)
    );
    """
]

drop_statements = [
    # "DROP TABLE IF EXISTS headlines.Articles;",
    # "DROP TABLE IF EXISTS headlines.Company_Info_News;",
    # "DROP TABLE IF EXISTS headlines.Pricing_News;",
    # "DROP TABLE IF EXISTS headlines.Volume_News;",
    # "DROP TABLE IF EXISTS headlines.Market_Data_Daily_Processing;",
    # "DROP TABLE IF EXISTS headlines.Market_Data_Headlines;",
    # "DROP TABLE IF EXISTS headlines.Trading_Calendar;",
    # "DROP TABLE IF EXISTS headlines.Articles_Trading_Day;",
    # "DROP TABLE IF EXISTS headlines.Market_Article_Summary;",
    # "DROP TABLE IF EXISTS headlines.Daily_Price_Movement;",
    # "DROP TABLE IF EXISTS headlines.Weekly_Price_Movement;",
    "DROP TABLE IF EXISTS headlines.extreme_price_movements;",
    "DROP TABLE IF EXISTS headlines.articles_extreme_drops;"
]

index_statements = [
    # "CREATE INDEX IF NOT EXISTS idx_articles_pubDate ON headlines.Articles (article_pubDate);",
    # "CREATE INDEX IF NOT EXISTS idx_articles_pubDate ON headlines.Articles_Trading_Day (article_pubDate);"
    # "CREATE INDEX IF NOT EXISTS idx_stock_movement_ticker ON headlines.Daily_Price_Movement (ticker);",
    # "CREATE INDEX IF NOT EXISTS idx_stock_movement_ticker ON headlines.Weekly_Price_Movement (ticker);"
]

for drop in drop_statements:
    con.execute(drop)

for ddl in ddl_statements:
    con.execute(ddl)

for index in index_statements:
    con.execute(index)


## Load `Company_Info_News`

In [ ]:
# its all in one line
with open(company_txt_path, 'r') as file:
    lines = file.readline().split('\\n')
    # con.execute("TRUNCATE Company_Info_News")
    for line in lines[1:]:
        line = line.strip().split('|')
        # DONT RUN THIS TWICE BY MISTAKE!
        con.execute("INSERT INTO headlines.Company_Info_News VALUES (?,?,?)", line)

## Load `Volume_News` 

In [ ]:
df = pd.read_csv(volume_news_path)
# df.head()

# convert the wide format to long format
volume_long_df = df.melt(id_vars=['Date'], var_name='Ticker', value_name='Volume')

# make sure they have the correct data types
volume_long_df['Date'] = pd.to_datetime(volume_long_df['Date'])
volume_long_df['Volume'] = pd.to_numeric(volume_long_df['Volume'], errors='coerce')

# con.execute("TRUNCATE Volume_News")
con.execute("INSERT INTO headlines.Volume_News (trading_day_date, ticker, Volume) SELECT Date, ticker, Volume FROM volume_long_df")

## Load `Pricing_News`

In [ ]:
df = pd.read_csv(pricing_news_path)
# convert the wide format to long format
pricing_long_df = df.melt(id_vars=['Date'], var_name='Ticker', value_name='Price')

# make sure they have the correct data types
pricing_long_df['Date'] = pd.to_datetime(pricing_long_df['Date'])
pricing_long_df['Price'] = pd.to_numeric(pricing_long_df['Price'], errors='coerce')

# con.execute("TRUNCATE Pricing_News")
con.execute("INSERT INTO headlines.Pricing_News (trading_day_date, Ticker, price) SELECT Date, ticker, Price FROM pricing_long_df")

## Load `Market_Data_Daily_Processing`

In [ ]:
# con.execute("TRUNCATE Market_Data_Daily_Processing")
con.execute("""
INSERT INTO headlines.Market_Data_Daily_Processing
SELECT 
    pn.trading_day_date,
    pn.ticker,
    pn.price,
    vn.volume
FROM 
    headlines.Pricing_News pn
LEFT JOIN 
    headlines.Volume_News vn 
ON 
    pn.trading_day_date = vn.trading_day_date AND pn.ticker = vn.ticker
""")

## Load `Trading_Calendar`

In [ ]:
# pricing_dates = pricing_long_df['Date'].drop_duplicates()
# volume_dates = volume_long_df['Date'].drop_duplicates()

# trading_dates = pd.concat([pricing_dates, volume_dates]).drop_duplicates().sort_values()

# # make sure to have correct col name
# trading_dates_df = pd.DataFrame(trading_dates, columns=['trading_date'])
# con.execute("TRUNCATE Trading_Calendar")
con.execute(
"""
INSERT INTO headlines.Trading_Calendar
SELECT DISTINCT trading_day_date AS trading_date
FROM (
    SELECT trading_day_date FROM headlines.Pricing_News
    UNION
    SELECT trading_day_date FROM headlines.Volume_News
) AS all_dates
ORDER BY trading_date;
"""
)

## Load `Articles`

In [ ]:
def xml_loader(base_dir):
    # lets do this in chunks instead
    failed_parses = pd.DataFrame()

    for root, dirs, files in os.walk(base_dir):
        data = [] 
        # extract ticker from foldername 
        ticker = os.path.basename(root)

        for file in files:
            file_path = os.path.join(root, file)
            
            try:
                tree = ET.parse(file_path)
                root_element = tree.getroot()
                
                channel = root_element.find('channel')
                if channel is not None:
                    # extract metadata info
                    language = channel.findtext("language") 
                    lastBuildDate = channel.findtext("lastBuildDate")
                    link = channel.findtext("link")
                    title = channel.findtext("title")
                    
                    # now meat and potatoes
                    for item in channel.findall("item"):
                        description = item.findtext("description")
                        guid = item.findtext("guid")
                        article_link = item.findtext("link")
                        article_pubDate = item.findtext("pubDate")
                        article_title = item.findtext("title")
                        
                        data.append({
                            "guid": guid,
                            "ticker": ticker,
                            "description": description,
                            "article_link": article_link,
                            "article_pubDate": article_pubDate,
                            "article_title": article_title,
                            "language": language,
                            "lastBuildDate": lastBuildDate,
                            "link": link,
                            "title": title
                        })
            except ET.parseError as e:
                print(f"Error parsing file {file_path}: {e}")
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")
        
        # insert the data into the database
        if data:
            df = pd.DataFrame(data)
            # print("Performing timestamp coercion for", ticker)
            df['parsed_date'] = pd.to_datetime(df['article_pubDate'], errors='coerce')
            df['lastBuildDate'] = pd.to_datetime(df['lastBuildDate'], errors='coerce')
            # print("Done timestamp coercion for", ticker)
            
            # separate failed cases to avoid nulls
            current_failed = df[df['parsed_date'].isna()]
            current_valid = df[df['parsed_date'].notna()]

            # these are good
            current_valid = current_valid.assign(article_pubDate=current_valid['parsed_date']).drop(columns=['parsed_date'])

            # remove dupes on guid and ticker
            current_valid.drop_duplicates(subset=['guid', 'ticker'], inplace=True)

            
            failed_parses = pd.concat([failed_parses, current_failed], ignore_index=True)
            
            try:
                # adding this too just in case
                con.execute("INSERT INTO headlines.Articles SELECT * FROM current_valid ON CONFLICT (guid, ticker) DO NOTHING")
                print("inserted data for", ticker)
            except Exception as e:
                print(f"Error inserting data for {ticker}: {e}")
                
    failed_parses.to_csv("failed_article_dates.csv", index=False)
    return failed_parses

In [ ]:
# GETTING NULLS! gonna fix the coercion logic
# con.execute("truncate Articles")

# load multicap headlines
failed_df = xml_loader(multicap_headlines)
# load new headlines
failed2_df = xml_loader(headline_august24_path)

In [ ]:
failed_df.head()

In [ ]:
# try again with the faulty data 
failed_df['article_pubDate'] = pd.to_datetime(failed_df['article_pubDate'], errors='coerce')
failed_df = failed_df.drop(columns=['parsed_date'])
failed_df.drop_duplicates(subset=['guid', 'ticker'], inplace=True)
try:
    # adding this too just in case
    con.execute("INSERT INTO headlines.Articles SELECT * FROM failed_df ON CONFLICT (guid, ticker) DO NOTHING")
except Exception as e:
    print(f"Error inserting data: {e}")

In [ ]:
# try again with the faulty data 
failed2_df['article_pubDate'] = pd.to_datetime(failed2_df['article_pubDate'], errors='coerce')
failed2_df = failed2_df.drop(columns=['parsed_date'])
failed2_df.drop_duplicates(subset=['guid', 'ticker'], inplace=True)
try:
    # adding this too just in case
    con.execute("INSERT INTO headlines.Articles SELECT * FROM failed2_df ON CONFLICT (guid, ticker) DO NOTHING")
except Exception as e:
    print(f"Error inserting data: {e}")

## Load `Articles_Trading_Day`

In [ ]:
con.execute("truncate headlines.Articles_Trading_Day")
con.execute("""
INSERT INTO headlines.Articles_Trading_Day
SELECT 
    a.guid,
    a.ticker,
    coalesce(MIN(tc.trading_date), cast(a.article_pubDate as Date)) AS mapped_trading_date,
    a.description,
    a.article_link,
    a.article_pubDate,
    a.article_title,
    a.language,
    a.lastBuildDate,
    a.link,
    a.title
FROM (
    SELECT 
        guid,
        ticker,
        description,
        article_link,
        article_pubDate,
        article_title,
        language,
        lastBuildDate,
        link,
        title,
        -- 4 PM EST adjust
        CASE 
            WHEN CAST(article_pubDate AS TIME) >= '16:00:00' 
            THEN CAST(article_pubDate AS DATE) + INTERVAL '1 day'
            ELSE CAST(article_pubDate AS DATE)
        END AS adjusted_pub_date
    FROM headlines.Articles
) a
LEFT JOIN 
    headlines.Trading_Calendar tc
ON 
    tc.trading_date >= a.adjusted_pub_date
GROUP BY 
    a.guid, a.ticker, a.description, a.article_link, a.article_pubDate, 
    a.article_title, a.language, a.lastBuildDate, a.link, a.title;
""")

## Create `Market_Data_Headlines`

In [ ]:
# con.execute("Truncate Market_Data_Headlines")
con.execute("""
INSERT INTO headlines.Market_Data_Headlines
SELECT 
    md.trading_day_date,
    md.ticker,
    md.price,
    md.volume,
    COALESCE(COUNT(DISTINCT atd.guid), 0) AS headline_count
FROM 
    headlines.Market_Data_Daily_Processing md
LEFT JOIN 
    headlines.Articles_Trading_Day atd
ON 
    md.ticker = atd.ticker AND md.trading_day_date = atd.mapped_trading_date
GROUP BY 
    md.trading_day_date, md.ticker, md.price, md.volume;
""")

## Load `market_article_summary`

In [ ]:
# con.execute("drop table headlines.Market_Article_Summary")
# con.execute("""
#                 CREATE TABLE IF NOT EXISTS headlines.Market_Article_Summary (
#     trading_date DATE PRIMARY KEY,
#     article_count INT
#     );

#             """)
con.execute('''
INSERT INTO headlines.Market_Article_Summary
SELECT 
    atd.mapped_trading_date AS trading_date,
    COUNT(DISTINCT atd.guid) AS total_unique_articles
FROM 
    headlines.Articles_Trading_Day atd
GROUP BY 
    atd.mapped_trading_date;
''')

## Load `Daily_Price_Movement`

In [ ]:
con.execute("truncate headlines.Daily_Price_Movement")
con.execute("""
            INSERT INTO headlines.Daily_Price_Movement
            SELECT 
                sp1.trading_day_date AS trading_date,
                sp1.ticker,
                sp1.price AS close_price,
                sp2.trading_day_date AS next_trading_day,
                sp2.price AS close_price_next,
                ROUND(sp2.price - sp1.price, 2) AS price_change,
                ROUND((sp2.price - sp1.price) / sp1.price * 100, 2) AS price_change_percentage
            FROM headlines.market_data_daily_processing sp1
            LEFT JOIN headlines.market_data_daily_processing sp2 
            ON sp2.ticker = sp1.ticker 
            AND sp2.trading_day_date = (
                SELECT MIN(sp3.trading_day_date) 
                FROM headlines.market_data_daily_processing sp3
                WHERE sp3.ticker = sp1.ticker
                AND sp3.trading_day_date > sp1.trading_day_date
            );
""")

## Load `Weekly_Price_Movement`

In [ ]:
con.execute("truncate headlines.Weekly_Price_Movement")
con.execute("""
            INSERT INTO headlines.Weekly_Price_Movement
            WITH WeeklyPrices AS (
            SELECT 
                ticker,
                MIN(tc.trading_date) AS trading_week_start,
                MAX(tc.trading_date) AS trading_week_end
            FROM headlines.market_data_daily_processing mdp
            JOIN headlines.trading_calendar tc 
            ON mdp.trading_day_date = tc.trading_date
            WHERE EXTRACT(DOW FROM tc.trading_date) BETWEEN 1 AND 5  -- Only weekdays
            GROUP BY ticker, DATE_TRUNC('week', tc.trading_date)
        ),
        StartPrices AS (
            SELECT 
                mdp.trading_day_date AS trading_week_start, 
                mdp.ticker, 
                mdp.price AS close_price
            FROM headlines.market_data_daily_processing mdp
            JOIN WeeklyPrices wp 
            ON mdp.ticker = wp.ticker 
            AND mdp.trading_day_date = wp.trading_week_start
        ),
        EndPrices AS (
            SELECT 
                mdp.trading_day_date AS trading_week_end, 
                mdp.ticker, 
                mdp.price AS close_price_end
            FROM headlines.market_data_daily_processing mdp
            JOIN WeeklyPrices wp 
            ON mdp.ticker = wp.ticker 
            AND mdp.trading_day_date = wp.trading_week_end
        )
        SELECT 
            sp.trading_week_start,
            sp.ticker,
            sp.close_price as close_price_start,
            ep.trading_week_end,
            ep.close_price_end,
            ROUND(ep.close_price_end - sp.close_price, 2) AS price_change,
            ROUND((ep.close_price_end - sp.close_price) / sp.close_price * 100, 2) AS price_change_percentage
        FROM StartPrices sp
        JOIN EndPrices ep 
        ON sp.ticker = ep.ticker 
        AND sp.trading_week_start = ep.trading_week_end - INTERVAL '4 days';
            """)

## Load `extreme_price_movements`

In [ ]:
con.execute("truncate headlines.extreme_price_movements")
con.execute("""
            INSERT INTO headlines.extreme_price_movements
            SELECT trading_date, ticker, close_price, price_change, price_change_percentage,
                CASE 
                    WHEN price_change_percentage < -5 THEN 'Drop'
                    WHEN price_change_percentage > 5 THEN 'Surge'
                END AS movement_type
            FROM headlines.daily_price_movement
            WHERE ABS(price_change_percentage) > 5;
""")

## Load `articles_extreme_drops`

In [ ]:
con.execute("truncate headlines.articles_extreme_drops")
df = con.execute("""
            SELECT epm.trading_date, epm.ticker, a.guid, a.mapped_trading_date,
                fs.finbert_title_score AS title_sentiment_score,
                fs.finbert_title_label AS title_sentiment_label,
                fs.finbert_description_score AS descripton_sentiment_score,
                fs.finbert_description_label AS descripton_sentiment_label
            FROM headlines.extreme_price_movements epm
            JOIN headlines.articles_trading_day a
            ON epm.ticker = a.ticker
            AND a.mapped_trading_date BETWEEN epm.trading_date - INTERVAL '3 days' AND epm.trading_date
            LEFT JOIN headlines.finbert_sentiment fs
            ON a.guid = fs.guid
""").df() 

# dedupe based on trading_date, ticker, guid 
df.drop_duplicates(subset=['trading_date', 'ticker', 'guid'], inplace=True)

con.execute("INSERT INTO headlines.articles_extreme_drops select * from df ON CONFLICT (trading_date, ticker, guid) DO NOTHING")

In [ ]:
con.close()

# SP500 Schema

In [ ]:
con = duckdb.connect("financial_news.db")
sp500_volume_weekly_path = 'SP500\\volume.csv'
sp500_price_weekly_path = 'SP500\\price.csv'
sp500_price_daily_path = 'SP500\\price_daily.csv'
sp500_company_path = 'SP500\\company_info_sp500.txt'
sp500_price_sp500_path = 'SP500\\price_SP500.csv'
sp500_item1_path = 'SP500\\sp500_item1_sec_filings_0.txt'
sp500_item1a_path = 'SP500\\sp500_item1a_sec_filings_0.txt'
sp500_item7_path = 'SP500\\sp500_item7_sec_filings_0.txt'

In [ ]:
# drop table statements 
drop_statements = [
    "DROP TABLE IF EXISTS SP500.Volume_Weekly;",
    "DROP TABLE IF EXISTS SP500.Price_Daily;",
    "DROP TABLE IF EXISTS SP500.Company_Info;",
    "DROP TABLE IF EXISTS SP500.Weekly_Market_Data;",
    "DROP TABLE IF EXISTS SP500.Price_Weekly;",
    "DROP TABLE IF EXISTS SP500.Price_Weekly_SP500;",
    "DROP TABLE IF EXISTS SP500.item7;",
    "DROP TABLE IF EXISTS SP500.item1a;",
    "DROP TABLE IF EXISTS SP500.item1;",
    "DROP TABLE IF EXISTS SP500.SEC_Item_Filings;"
]

ddl_statements = [
    "CREATE SCHEMA IF NOT EXISTS SP500;",
"""
CREATE TABLE IF NOT EXISTS SP500.Volume_Weekly (
    trading_week_date DATE NOT NULL,
    cik TEXT NOT NULL,
    volume FLOAT,
    PRIMARY KEY (trading_week_date, cik)
);
""",
"""
CREATE TABLE IF NOT EXISTS SP500.Price_Daily (
    trading_day_date DATE NOT NULL,
    cik TEXT NOT NULL,
    price FLOAT,
    PRIMARY KEY (trading_day_date, cik)
);
""",
"""
CREATE TABLE IF NOT EXISTS SP500.Company_Info (
    cik TEXT PRIMARY KEY,
    ticker TEXT,  
    name TEXT, 
    subindustry TEXT 
);
""",
"""
CREATE TABLE IF NOT EXISTS SP500.Weekly_Market_Data (
    trading_week_date DATE NOT NULL,
    cik TEXT NOT NULL,
    price FLOAT NOT NULL,
    volume FLOAT NOT NULL,
    PRIMARY KEY (trading_week_date, cik)
);
""",
"""
CREATE TABLE IF NOT EXISTS SP500.Price_Weekly (
    trading_week_date DATE NOT NULL,
    cik TEXT NOT NULL,
    price FLOAT,
    PRIMARY KEY (trading_week_date, cik)
);
""",
"""
CREATE TABLE IF NOT EXISTS SP500.Price_Weekly_SP500 (
    trading_week_date DATE PRIMARY KEY,
    SP500CapWeighted FLOAT,
    SP500EqualWeighted FLOAT
);
""",
"""
CREATE TABLE IF NOT EXISTS SP500.item7 (
    company TEXT NOT NULL,           
    filing_ts TIMESTAMP NOT NULL,              
    link TEXT,                       
    type TEXT,                       
    cik TEXT,                         
    item7 TEXT,                      
    PRIMARY KEY (company, filing_ts)      
);
""",
"""
CREATE TABLE IF NOT EXISTS SP500.item1a (
    company TEXT NOT NULL,           
    filing_ts TIMESTAMP NOT NULL,              
    link TEXT,                       
    type TEXT,                       
    cik TEXT,                         
    item1a TEXT,                     
    PRIMARY KEY (company, filing_ts)      
);
""",
"""
CREATE TABLE IF NOT EXISTS SP500.item1 (
    company TEXT NOT NULL,           
    filing_ts TIMESTAMP NOT NULL,              
    link TEXT,                       
    type TEXT,                       
    cik TEXT,                         
    item1 TEXT,                      
    PRIMARY KEY (company, filing_ts)      
);
""",
"""
CREATE TABLE IF NOT EXISTS SP500.SEC_Item_Filings (
    cik TEXT NOT NULL,               
    filing_ts TIMESTAMP NOT NULL,              
    item_filing TEXT NOT NULL,       
    company TEXT,                    
    link TEXT,                       
    type TEXT,                       
    item_description TEXT,           
    PRIMARY KEY (cik, filing_ts, item_filing)
);
"""
]

for drop in drop_statements:
    con.execute(drop)

for ddl in ddl_statements:
    con.execute(ddl)

# Load `volume_weekly`

In [ ]:
df = pd.read_csv(sp500_volume_weekly_path)
# convert the wide format to long format
# volume_long_df = wide_to_long(df, ['Date'], 'Volume', 'cik')
volume_long_df = df.melt(id_vars=['Date'], var_name='cik', value_name='Volume')

# make sure they have the correct data types
volume_long_df['Date'] = pd.to_datetime(volume_long_df['Date'])
volume_long_df['Volume'] = pd.to_numeric(volume_long_df['Volume'], errors='coerce')

# default null volume values to 0. CIK 1534701 is all nulls so better to just drop the column but keep it for now
# nah jk leaving it as null for now. will coalesce the final table
# volume_long_df['Volume'] = volume_long_df['Volume'].fillna(0)

con.execute("INSERT INTO SP500.Volume_Weekly (trading_week_date, cik, volume) SELECT date, cik, volume FROM volume_long_df")

# Load `Price_Daily`

In [ ]:
df = pd.read_csv(sp500_price_daily_path)
# convert the wide format to long format
# price_long_df = wide_to_long(df, ['Date'], 'Price', 'cik')
price_long_df = df.melt(id_vars=['Date'], var_name='cik', value_name='Price')

# make sure they have the correct data types
price_long_df['Date'] = pd.to_datetime(price_long_df['Date'])
price_long_df['Price'] = pd.to_numeric(price_long_df['Price'], errors='coerce')

# con.execute("TRUNCATE SP500.Price_Daily")
con.execute("INSERT INTO SP500.Price_Daily (trading_day_date, cik, price) SELECT Date, cik, Price FROM price_long_df")

In [ ]:
price_long_df

# Load `Price_Weekly`

<b>Looks like theres an issue with this dataset. When the office hours recordings come out I'll watch it and fix it. until then it'll be null </b>

In [ ]:
df.head()

In [ ]:
df = pd.read_csv(sp500_price_weekly_path)
# convert the wide format to long format
# price_long_df = wide_to_long(df, ['Date'], 'Price', 'cik')
price_long_df = df.melt(id_vars=['Date'], var_name='cik', value_name='Price')

# make sure they have the correct data types
price_long_df['Date'] = pd.to_datetime(price_long_df['Date'])
price_long_df['Price'] = pd.to_numeric(price_long_df['Price'], errors='coerce')

# con.execute("TRUNCATE SP500.Price_Weekly")
# con.execute("INSERT INTO SP500.Price_Weekly (trading_week_date, cik, price) SELECT Date, cik, price FROM price_long_df")

# Load `Company_Info`

In [ ]:
# its all in one line
with open(sp500_company_path, 'r') as file:
    lines = file.readline().split('\\n')
    for line in lines[1:]:
        line = line.strip().split('|')
        # DONT RUN THIS TWICE BY MISTAKE!
        con.execute("INSERT INTO SP500.Company_Info VALUES (?,?,?,?)", line)

# Load `Weekly_Market_Data`

In [ ]:
con.execute("""
INSERT INTO SP500.Weekly_Market_Data
SELECT 
    pw.trading_week_date AS trading_week_date,
    pw.cik AS cik,
    coalesce(pw.price, 0) AS price,
    coalesce(vw.volume, 0) AS volume
FROM 
    SP500.Price_Weekly pw
LEFT JOIN 
    SP500.Volume_Weekly vw
ON 
    pw.trading_week_date = vw.trading_week_date AND pw.cik = vw.cik
""")

# Load `SP500.Price_Weekly_SP500`

In [ ]:
df = pd.read_csv(sp500_price_sp500_path)

df['Date'] = pd.to_datetime(df['Date'])
df['SP500CapWeighted'] = pd.to_numeric(df['SP500CapWeighted'], errors='coerce')
df['SP500EqualWeighted'] = pd.to_numeric(df['SP500EqualWeighted'], errors='coerce')

con.execute("truncate SP500.Price_Weekly_SP500")
con.execute("INSERT INTO SP500.Price_Weekly_SP500 (trading_week_date,SP500CapWeighted, SP500EqualWeighted) SELECT Date, SP500CapWeighted, SP500EqualWeighted FROM df")

# Load `item7 `

Sometimes the last columns comes in multiple lines. Sucks b/c its last column and I can't rely on the pipe, so gotta code for that 

In [ ]:
# cols: company|date|link|type|cik|item7 
# con.execute("TRUNCATE SP500.item7")
def parse_items(file_path, table_name):
    # use 2 pointer approach to check if next line is a continuation of the current line
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        prev_line = None  

        for line in lines[1:]:
            line = line.strip()
            fields = line.split('|')

            if prev_line is None:
                prev_line = fields
                continue

            # if current line has required number of elements, insert prev line
            if len(fields) == 6:
                con.execute(f"INSERT INTO SP500.{table_name} VALUES (?,?,?,?,?,?)", prev_line)
                prev_line = fields
            else:
                prev_line[-1] += " " + line

        # the last record
        if prev_line:
            con.execute(f"INSERT INTO SP500.{table_name} VALUES (?,?,?,?,?,?)", prev_line)

In [ ]:
con.execute("TRUNCATE SP500.item7")
parse_items(sp500_item7_path, 'item7')

# Load `item1`

In [ ]:
con.execute("TRUNCATE SP500.item1")
parse_items(sp500_item1_path, 'item1')

# Load `item1a`

In [ ]:
con.execute("TRUNCATE SP500.item1a")
parse_items(sp500_item1a_path, 'item1a')

# Load `SEC_Item_Filings`

I think the only thing different between the 3 item tables are the item filing # and the item description

With that said, I believe its better to use a longer table for simplicity vs a wider table 
- wide = instead of 1 item_filing and 1 item_description columns, we make a column for each filing and description

In [ ]:
con.execute("""
INSERT INTO SP500.SEC_Item_Filings
SELECT 
    cik,
    filing_ts,
    '7' AS item_filing,
    company,
    link,
    type,
    item7 AS item_description
FROM 
    SP500.item7
UNION ALL
SELECT 
    cik,
    filing_ts,
    '1a' AS item_filing,
    company,
    link,
    type,
    item1a AS item_description
FROM 
    SP500.item1a
UNION ALL
SELECT 
    cik,
    filing_ts,
    '1' AS item_filing,
    company,
    link,
    type,
    item1 AS item_description
FROM 
    SP500.item1;
""")

In [ ]:
con.close()

# FinBERT related stuff

## FinBERT Scores

In [ ]:
con = duckdb.connect("financial_news.db")
finbert_csv = "articles_with_all_finbert_scores.csv"
# SO THIS ONE WILL HAVE DUPES! NEED TO DEDUPE!
tokens_csv = "tokens.csv"

In [ ]:
ddl = ["""
CREATE TABLE IF NOT EXISTS headlines.finbert_sentiment (
    guid TEXT,
    description TEXT,
    article_title TEXT,
    ticker TEXT,
    finbert_title_label TEXT,
    finbert_title_score FLOAT,
    finbert_description_label TEXT,
    finbert_description_score FLOAT,
    PRIMARY KEY (guid, ticker)
);
""",
"""
CREATE TABLE IF NOT EXISTS headlines.tokens_description (
    guid TEXT,
    token TEXT,
    token_lemmatized TEXT,
    frequency INT,
    PRIMARY KEY (guid, token, token_lemmatized)
);
""",
"""
CREATE TABLE IF NOT EXISTS headlines.tokens_title (
    guid TEXT,
    token TEXT,
    token_lemmatized TEXT,
    frequency INT,
    PRIMARY KEY (guid, token, token_lemmatized)
);
"""
]

drop = [
    "DROP TABLE IF EXISTS headlines.finbert_sentiment;",
    "DROP TABLE IF EXISTS headlines.tokens_description;",
    "DROP TABLE IF EXISTS headlines.tokens_title;"
    ]

for d in drop:
    con.execute(d)
    
for d in ddl:
    con.execute(d)

# Load `finbert_sentiment`

In [ ]:
df = pd.read_csv(finbert_csv)
# df.head()
filtered_df = df[['guid', 'description', 'article_title', 'ticker', 'finbert_title_label', 'finbert_title_score', 'finbert_description_label', 'finbert_description_score']]
filtered_df.head()

In [ ]:
con.execute("truncate headlines.finbert_sentiment")
con.execute("INSERT INTO headlines.finbert_sentiment select * from filtered_df")

# Load `tokens_description`

In [ ]:
# SO THIS ONE WILL HAVE DUPES! NEED TO DEDUPE!
df = pd.read_csv(tokens_csv)
print(df.size)

# remove df duplicates based on guid 
df.drop_duplicates(subset=['guid'], inplace=True)
print(df.size)
df.head()

In [ ]:
tokens_description_data = []
tokens_title_data = []

for _, row in df.iterrows():
    guid = row['guid']
    
    tokens_description = eval(row['tokens_description'])
    tokens_title = eval(row['tokens_title'])
    
    for token, lemma  in tokens_description:
        tokens_description_data.append((guid, token, lemma))
    
    for token, lemma  in tokens_title:
        tokens_title_data.append((guid, token, lemma))

df_tokens_description = pd.DataFrame(tokens_description_data, columns=["guid", "token", "token_lemmatized"])
df_tokens_title = pd.DataFrame(tokens_title_data, columns=["guid", "token", "token_lemmatized"])

In [ ]:
# df_tokens_description.head()
# find guid 367bed80-8d07-3dce-8092-fd53d70578fe with token quarter
# df_tokens_description[(df_tokens_description['guid'] == '367bed80-8d07-3dce-8092-fd53d70578fe' ) & (df_tokens_description['token'] == 'quarter')]
# aggregate any duplicates and count them and add them to column frequency
### OKAY LETS KEEP token_lemmatized HERE BC LEMMAS RELY ON CONTEXT! 
df_tokens_description = df_tokens_description.groupby(['guid', 'token', 'token_lemmatized']).size().reset_index(name='frequency')
# df_tokens_title = df_tokens_title.groupby(['guid', 'token', 'token_lemmatized']).size().reset_index(name='frequency')
df_tokens_description.head()

In [ ]:
# 000705ff-4abf-355e-bed9-aeb6733f92b3 with token spending 
df_tokens_description[(df_tokens_description['guid'] == '000705ff-4abf-355e-bed9-aeb6733f92b3' ) & (df_tokens_description['token'] == 'spending')].head()

In [ ]:
con.execute("truncate headlines.tokens_description")
con.execute("INSERT INTO headlines.tokens_description select * from df_tokens_description")

# Load `tokens_title`

In [ ]:
df_tokens_title = df_tokens_title.groupby(['guid', 'token', 'token_lemmatized']).size().reset_index(name='frequency')
df_tokens_title.head()

In [ ]:
con.execute("truncate headlines.tokens_title")
con.execute("INSERT INTO headlines.tokens_title select * from df_tokens_title")

In [ ]:
con.close()

# 10k FinBERT

In [ ]:
con = duckdb.connect("financial_news.db")
finbert_csv = "sp500_with_all_finbert_scores.csv"
# # SO THIS ONE WILL HAVE DUPES! NEED TO DEDUPE!
# tokens_csv = "tokens.csv"

In [ ]:
ddl = [
    """
    CREATE TABLE if not exists sp500.SEC_Item_Filings_FinBERT (
    cik TEXT,
    filing_ts TIMESTAMP,
    item_filing TEXT,
    finbert_description_label TEXT,
    finbert_description_score FLOAT,
    PRIMARY KEY (cik, filing_ts, item_filing),
    FOREIGN KEY (cik, filing_ts, item_filing) 
        REFERENCES sp500.SEC_Item_Filings (cik, filing_ts, item_filing) 
    );
    """
]

drop = [
    "DROP TABLE IF EXISTS sp500.SEC_Item_Filings_FinBERT;"
]

for d in drop:
    con.execute(d)
    
for d in ddl:
    con.execute(d)



# SEC_Item_Filings_FinBERT

In [ ]:
finbert_10k_df = pd.read_csv(finbert_csv)
finbert_10k_df.head()

In [ ]:
finbert_10k_df = finbert_10k_df[['cik', 'filing_ts', 'item_filing', 'finbert_description_label', 'finbert_description_score']]

In [ ]:
con.execute("truncate sp500.SEC_Item_Filings_FinBERT")
con.execute("INSERT INTO sp500.SEC_Item_Filings_FinBERT select * from finbert_10k_df")

In [ ]:
con.close()

# VIX DATASET

In [ ]:
con = duckdb.connect("financial_news.db")
vix = "SP500/vixGaTechSP25.csv"

In [ ]:
ddl = [
    """
    CREATE TABLE sp500.VIX_Index (
    vix_date DATE PRIMARY KEY,
    vix_value FLOAT
);
    """
]

drop = [
    "DROP TABLE IF EXISTS sp500.VIX_Index;"
]

for d in drop:
    con.execute(d)
    
for d in ddl:
    con.execute(d)

## Load `VIX_Index`

In [ ]:
vix_df = pd.read_csv(vix, names=["vix_date", "vix_value"], parse_dates=["vix_date"], skiprows=1) # first row is the header but not the best
vix_df.head()

In [ ]:
# check for nulls 
vix_df.isnull().sum()

In [ ]:
con.execute("Truncate sp500.VIX_Index")
con.execute("INSERT INTO sp500.VIX_Index select * from vix_df")

In [ ]:
con.close()

# VIX Preds

In [7]:
con = duckdb.connect(r"C:/Users/jovan/Documents/CSE 6748 Practicum WITH DATA/financial_news.db")
vix_daily_res = "daily_results.csv"
vix_weekly_res = "weekly_results.csv"
vix_daily_train = "daily_train_df.csv"
vix_weekly_train = "weekly_train_df.csv"

In [8]:
ddls = [
"""

    CREATE TABLE IF NOT EXISTS sp500.rolling_predictions_daily (
        date DATE,
        actual_vix FLOAT,
        predicted_vix FLOAT,
        primary key (date)
    );

""", 
"""
    CREATE TABLE IF NOT EXISTS sp500.rolling_predictions_weekly (
        date DATE,
        actual_vix FLOAT,
        predicted_vix FLOAT,
        primary key (date)
    );
""",
# make ddl for "vix_date","vix_value"
"""
CREATE TABLE IF NOT EXISTS sp500.vix_daily_training (
    vix_date DATE PRIMARY KEY,
    vix_value FLOAT
);
""",
"""
Create Table if not exists sp500.vix_weekly_training (
    vix_date DATE PRIMARY KEY,
    vix_value FLOAT
);
"""
]

drops = [
    # "DROP TABLE IF EXISTS sp500.rolling_predictions_daily;",
    # "DROP TABLE IF EXISTS sp500.rolling_predictions_weekly;"
    "DROP TABLE IF EXISTS sp500.vix_daily_training;",
    "DROP TABLE IF EXISTS sp500.vix_weekly_training;"
]

for drop in drops:
    con.execute(drop)
    
for ddl in ddls:
    con.execute(ddl)

## Load `rolling_predictions_daily`

In [ ]:
df = pd.read_csv(vix_daily_res)
con.execute("truncate sp500.rolling_predictions_daily")
con.execute("INSERT INTO sp500.rolling_predictions_daily select * from df")

## Load `rolling_predictions_weekly`

In [ ]:
df = pd.read_csv(vix_weekly_res)
con.execute("truncate sp500.rolling_predictions_weekly")
con.execute("INSERT INTO sp500.rolling_predictions_weekly select * from df")

## `sp500.vix_daily_training`

In [9]:
df = pd.read_csv(vix_daily_train)
con.execute("truncate sp500.vix_daily_training")
con.execute("INSERT INTO sp500.vix_daily_training select * from df")

## `sp500.vix_weekly_training`

In [10]:
df = pd.read_csv(vix_weekly_train)
con.execute("truncate sp500.vix_weekly_training")
con.execute("INSERT INTO sp500.vix_weekly_training select * from df")

In [11]:
con.close()

# FinBERT All Scores

In [ ]:
con = duckdb.connect(r"C:/Users/jovan/Documents/CSE 6748 Practicum WITH DATA/financial_news.db")
finbert_all_tags = "articles_with_all_finbert_scores.csv"

In [ ]:
ddl = ["""
CREATE TABLE IF NOT EXISTS headlines.finbert_analysis (
    guid UUID ,
    ticker VARCHAR(10) NOT NULL,
    description TEXT,
    article_title TEXT,
    finbert_title_label VARCHAR(20) NOT NULL,
    finbert_title_score FLOAT NOT NULL,
    finbert_title_positive FLOAT NOT NULL,
    finbert_title_neutral FLOAT NOT NULL,
    finbert_title_negative FLOAT NOT NULL,
    finbert_description_label VARCHAR(20) NOT NULL,
    finbert_description_score FLOAT NOT NULL,
    finbert_description_positive FLOAT NOT NULL,
    finbert_description_neutral FLOAT NOT NULL,
    finbert_description_negative FLOAT NOT NULL,
    PRIMARY KEY (guid, ticker)
);

"""]

drop = [
    "DROP TABLE IF EXISTS headlines.finbert_analysis;"
]

for d in drop:
    con.execute(d)

for d in ddl:
    con.execute(d)

## `headlines.finbert_analysis`

In [ ]:
con.execute("truncate headlines.finbert_analysis")
df = pd.read_csv(finbert_all_tags)
con.execute("INSERT INTO headlines.finbert_analysis select guid, ticker, description, article_title, finbert_title_label, finbert_title_score, finbert_title_positive, finbert_title_neutral, finbert_title_negative, finbert_description_label, finbert_description_score, finbert_description_positive, finbert_description_neutral, finbert_description_negative from df")

In [ ]:
con.close()

## Training Data for VIX Model

In [ ]:
con = duckdb.connect(r"C:/Users/jovan/Documents/CSE 6748 Practicum WITH DATA/financial_news.db")

In [ ]:
ddl = [
    """
CREATE TABLE Headlines.weekly_training_data(guid VARCHAR,
date_t DATE,
ticker VARCHAR,
subindustry VARCHAR,
vix_t FLOAT,
vix_t_7_past FLOAT,
vix_t_7_future FLOAT,
price_t FLOAT,
price_t_7_past FLOAT,
price_change_t_7 FLOAT,
volume_t INTEGER,
volume_t_7_past INTEGER,
volume_change_t_7 DOUBLE,
sentiment_label_t VARCHAR,
sentiment_positive_t FLOAT,
sentiment_neutral_t FLOAT,
sentiment_negative_t FLOAT);
"""
]

drop = [
    "DROP TABLE IF EXISTS Headlines.weekly_training_data;"
]


for d in drop:
    con.execute(d)
    
for d in ddl:
    con.execute(d)

`Headlines.weekly_training_data`

In [ ]:
con.execute("truncate Headlines.weekly_training_data")
con.execute("""
            INSERT INTO Headlines.weekly_training_data
            SELECT * FROM (
WITH vix_lagged AS (
    SELECT 
        v1.vix_date AS date_t,
        v1.vix_value AS vix_t,
        COALESCE(LAG(v1.vix_value, 1) OVER (ORDER BY v1.vix_date), 17.22) AS vix_t_7_past, -- same add last val
        COALESCE(LEAD(v1.vix_value, 1) OVER (ORDER BY v1.vix_date), 23.39) AS vix_t_7_future -- adding to handle last vix day that we don't know
    FROM sp500.vix_weekly_training v1
)
--select * from vix_lagged;
,
-- ANYTHING WITH -1 in price_t, price_t7 or volume cols should be removed! 
market_lagged AS (
    SELECT 
        m1.trading_day_date AS date_t,
        m1.ticker,
        COALESCE(m1.price, -1) AS price_t,  -- Set -1 if all price data is NULL
        COALESCE(LAG(m1.price, 1) OVER (PARTITION BY m1.ticker ORDER BY m1.trading_day_date), -1) AS price_t_7_past,
        (CASE 
            WHEN LAG(m1.price, 1) OVER (PARTITION BY m1.ticker ORDER BY m1.trading_day_date) IS NOT NULL 
            THEN ((m1.price - LAG(m1.price, 1) OVER (PARTITION BY m1.ticker ORDER BY m1.trading_day_date)) / 
                  LAG(m1.price, 1) OVER (PARTITION BY m1.ticker ORDER BY m1.trading_day_date)) * 100
            ELSE NULL 
        END) AS price_change_t_7,
        COALESCE(m1.volume, -1) AS volume_t,  -- Set -1 if all volume data is NULL
        COALESCE(LAG(m1.volume, 1) OVER (PARTITION BY m1.ticker ORDER BY m1.trading_day_date), -1) AS volume_t_7_past,
        (CASE 
            WHEN LAG(m1.volume, 1) OVER (PARTITION BY m1.ticker ORDER BY m1.trading_day_date) IS NOT NULL 
            THEN ((m1.volume - LAG(m1.volume, 1) OVER (PARTITION BY m1.ticker ORDER BY m1.trading_day_date)) / 
                  NULLIF(LAG(m1.volume, 1) OVER (PARTITION BY m1.ticker ORDER BY m1.trading_day_date), 0)) * 100
            ELSE NULL 
        END) AS volume_change_t_7
    FROM headlines.Market_Data_Headlines m1
--    where ticker in (
--    	select * from headlines.sp500_active_stocks
--    )
)
--select * from market_lagged;
,
article_sentiment AS (
    SELECT 
    	a.guid,
        a.mapped_trading_date AS date_t,
        a.ticker,
        f.finbert_description_positive AS sentiment_positive_t,
        f.finbert_description_neutral AS sentiment_neutral_t,
        f.finbert_description_negative AS sentiment_negative_t,
        f.finbert_description_label AS sentiment_label_t  -- Just default to NEUTRAL
    FROM Headlines.Articles_Trading_Day a
    JOIN Headlines.finbert_analysis f ON a.guid = f.guid
)
--select * from article_sentiment;
--select count(*) from (
SELECT distinct -- sometimes we have dupes...I think we have dupes upstream but oh whale
s.guid,
    v.date_t,
    m.ticker,
    c.subindustry,
    v.vix_t,
    v.vix_t_7_past,
    v.vix_t_7_future,
    m.price_t,
    m.price_t_7_past,
    m.price_change_t_7,
    m.volume_t,
    m.volume_t_7_past,
    m.volume_change_t_7,
    coalesce(s.sentiment_label_t, 'NEUTRAL') as sentiment_label_t,
    coalesce(s.sentiment_positive_t, 0) as sentiment_positive_t,
    coalesce(s.sentiment_neutral_t, 1) as sentiment_neutral_t,
    coalesce(s.sentiment_negative_t, 0) as sentiment_negative_t
FROM vix_lagged v
JOIN market_lagged m ON v.date_t = m.date_t
LEFT JOIN article_sentiment s ON v.date_t = s.date_t AND m.ticker = s.ticker
LEFT JOIN sp500.company_info c ON m.ticker = c.ticker  -- NEW JOIN
order by v.date_t desc) as insert_query;
""")

In [ ]:
con.close()